# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 27 2023 9:07AM,256057,10,0086333442,ISDIN Corporation,Released
1,Jan 27 2023 9:07AM,256057,10,0086333444,ISDIN Corporation,Released
2,Jan 27 2023 9:07AM,256057,10,0086333462,ISDIN Corporation,Released
3,Jan 27 2023 9:07AM,256057,10,0086333463,ISDIN Corporation,Released
4,Jan 27 2023 9:07AM,256057,10,0086333464,ISDIN Corporation,Released
5,Jan 27 2023 9:07AM,256057,10,0086333465,ISDIN Corporation,Released
6,Jan 27 2023 9:07AM,256057,10,0086333466,ISDIN Corporation,Released
7,Jan 27 2023 9:07AM,256057,10,0086333468,ISDIN Corporation,Released
8,Jan 27 2023 9:07AM,256057,10,0086333460,ISDIN Corporation,Released
9,Jan 27 2023 9:07AM,256057,10,0086333467,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,256054,Released,15
12,256055,Released,2
13,256056,Released,22
14,256057,Released,56
15,256058,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256054,NaN,NaN,15.0
256055,NaN,NaN,2.0
256056,NaN,NaN,22.0
256057,NaN,NaN,56.0
256058,NaN,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255969,0.0,1.0,0.0
255987,48.0,0.0,1.0
256028,0.0,0.0,1.0
256030,0.0,0.0,1.0
256031,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255969,0,1,0
255987,48,0,1
256028,0,0,1
256030,0,0,1
256031,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255969,0,1,0
1,255987,48,0,1
2,256028,0,0,1
3,256030,0,0,1
4,256031,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255969,,1,
1,255987,48,,1
2,256028,,,1
3,256030,,,1
4,256031,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 27 2023 9:07AM,256057,10,ISDIN Corporation
56,Jan 27 2023 9:05AM,256056,10,ISDIN Corporation
78,Jan 27 2023 9:01AM,256058,10,Emerginnova
87,Jan 27 2023 8:59AM,256055,10,ISDIN Corporation
89,Jan 27 2023 8:50AM,256054,10,ISDIN Corporation
104,Jan 27 2023 8:19AM,256050,16,WHC Labs bvba
105,Jan 27 2023 7:53AM,256046,19,"AdvaGen Pharma, Ltd"
147,Jan 26 2023 4:38PM,256042,19,ACG North America LLC
148,Jan 26 2023 3:43PM,256038,12,HealthCaps LLC
149,Jan 26 2023 3:25PM,256031,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 27 2023 9:07AM,256057,10,ISDIN Corporation,,,56
1,Jan 27 2023 9:05AM,256056,10,ISDIN Corporation,,,22
2,Jan 27 2023 9:01AM,256058,10,Emerginnova,,,9
3,Jan 27 2023 8:59AM,256055,10,ISDIN Corporation,,,2
4,Jan 27 2023 8:50AM,256054,10,ISDIN Corporation,,,15
5,Jan 27 2023 8:19AM,256050,16,WHC Labs bvba,,,1
6,Jan 27 2023 7:53AM,256046,19,"AdvaGen Pharma, Ltd",,19,23
7,Jan 26 2023 4:38PM,256042,19,ACG North America LLC,,,1
8,Jan 26 2023 3:43PM,256038,12,HealthCaps LLC,,,1
9,Jan 26 2023 3:25PM,256031,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 9:07AM,256057,10,ISDIN Corporation,56,,
1,Jan 27 2023 9:05AM,256056,10,ISDIN Corporation,22,,
2,Jan 27 2023 9:01AM,256058,10,Emerginnova,9,,
3,Jan 27 2023 8:59AM,256055,10,ISDIN Corporation,2,,
4,Jan 27 2023 8:50AM,256054,10,ISDIN Corporation,15,,
5,Jan 27 2023 8:19AM,256050,16,WHC Labs bvba,1,,
6,Jan 27 2023 7:53AM,256046,19,"AdvaGen Pharma, Ltd",23,19,
7,Jan 26 2023 4:38PM,256042,19,ACG North America LLC,1,,
8,Jan 26 2023 3:43PM,256038,12,HealthCaps LLC,1,,
9,Jan 26 2023 3:25PM,256031,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 9:07AM,256057,10,ISDIN Corporation,56,,
1,Jan 27 2023 9:05AM,256056,10,ISDIN Corporation,22,,
2,Jan 27 2023 9:01AM,256058,10,Emerginnova,9,,
3,Jan 27 2023 8:59AM,256055,10,ISDIN Corporation,2,,
4,Jan 27 2023 8:50AM,256054,10,ISDIN Corporation,15,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 9:07AM,256057,10,ISDIN Corporation,56.0,NaN,NaN
1,Jan 27 2023 9:05AM,256056,10,ISDIN Corporation,22.0,NaN,NaN
2,Jan 27 2023 9:01AM,256058,10,Emerginnova,9.0,NaN,NaN
3,Jan 27 2023 8:59AM,256055,10,ISDIN Corporation,2.0,NaN,NaN
4,Jan 27 2023 8:50AM,256054,10,ISDIN Corporation,15.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 9:07AM,256057,10,ISDIN Corporation,56.0,0.0,0.0
1,Jan 27 2023 9:05AM,256056,10,ISDIN Corporation,22.0,0.0,0.0
2,Jan 27 2023 9:01AM,256058,10,Emerginnova,9.0,0.0,0.0
3,Jan 27 2023 8:59AM,256055,10,ISDIN Corporation,2.0,0.0,0.0
4,Jan 27 2023 8:50AM,256054,10,ISDIN Corporation,15.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1280280,104.0,0.0,0.0
12,256038,1.0,0.0,0.0
15,255987,1.0,0.0,48.0
16,768108,3.0,0.0,0.0
19,512088,24.0,19.0,0.0
21,512000,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1280280,104.0,0.0,0.0
1,12,256038,1.0,0.0,0.0
2,15,255987,1.0,0.0,48.0
3,16,768108,3.0,0.0,0.0
4,19,512088,24.0,19.0,0.0
5,21,512000,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,104.0,0.0,0.0
1,12,1.0,0.0,0.0
2,15,1.0,0.0,48.0
3,16,3.0,0.0,0.0
4,19,24.0,19.0,0.0
5,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,104.0
1,12,Released,1.0
2,15,Released,1.0
3,16,Released,3.0
4,19,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,0.0,0.0,48.0,0.0,0.0,0.0
Executing,0.0,0.0,0.0,0.0,19.0,1.0
Released,104.0,1.0,1.0,3.0,24.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,48.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,19.0,1.0
2,Released,104.0,1.0,1.0,3.0,24.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,48.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,19.0,1.0
2,Released,104.0,1.0,1.0,3.0,24.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()